# Optimal portfolio

1. Connect to RH,
2. get securities, 
3. get value, 
4. calculate optimal portfolio, 
5. calculate diff, 
6. sell positions, 
7. buy positions, 
8. stop loss

Things to try:

- sample from large pool of securities, give equal weights, calculate sharpe ratio, maximize
- same as above but with exponentially smoothed sharpe ratios
- set stop loss to Close_ewm0.1_mean-1*Close_ewm0.1_std and initiate limit buy at Close_ewm0.1_mean

Alternative:

- take all V etfs, give random weights, calculate sharpe ratio, maximize


In [1]:
import pulp

ModuleNotFoundError: No module named 'pulp'

In [ ]:
def optimal_portfolio(df=None, price_col='Close'):

    # Create variables for all players
    QB_ID = df[df['Position'] == 'QB']['GID'].values.tolist()
    TE_ID = df[df['Position'] == 'TE']['GID'].values.tolist()
    RB_ID = df[df['Position'] == 'RB']['GID'].values.tolist()
    WR_ID = df[df['Position'] == 'WR']['GID'].values.tolist()
    DST_ID = df[df['Position'] == 'DST']['GID'].values.tolist()
    POS_ID = QB_ID + TE_ID + RB_ID + WR_ID + DST_ID

    # create dictionaries for salaries and performance
    x = LpVariable.dicts("%s", POS_ID, 0, 1, LpInteger)
    points = pd.Series(df[points_col].values, index=df['GID']).to_dict()
    salary = pd.Series(df['Salary'].values, index=df['GID']).to_dict()

    dk_solve = LpProblem("ILP", LpMaximize)

    # ****************************************************************
    # Objective
    # ****************************************************************
    dk_solve += sum([points[i] * x[i] for i in sorted(POS_ID)])

    # ****************************************************************
    # Constraints
    # ****************************************************************

    # Salary Cap at $50k
    dk_solve += sum([salary[i] * x[i] for i in sorted(POS_ID)]) <= 50000

    # Only 1 Quaterback
    dk_solve += sum([x[i] for i in sorted(QB_ID)]) == 1

    # Between 1 and 2 Tight Ends
    dk_solve += sum([x[i] for i in sorted(TE_ID)]) <= 2
    dk_solve += sum([x[i] for i in sorted(TE_ID)]) >= 1

    # Between 3 and 4 Wide Receivers
    dk_solve += sum([x[i] for i in sorted(WR_ID)]) <= 4
    dk_solve += sum([x[i] for i in sorted(WR_ID)]) >= 3
    # dk_solve += sum([x[i] for i in sorted(WR_ID)])  == 3

    # Between 2 and 3 Running Backs
    dk_solve += sum([x[i] for i in sorted(RB_ID)]) <= 3
    dk_solve += sum([x[i] for i in sorted(RB_ID)]) >= 2

    # Only 1 Defence / Special Teams
    dk_solve += sum([x[i] for i in sorted(DST_ID)]) == 1

    # Require 9 players
    dk_solve += sum([x[i] for i in sorted(POS_ID)]) == 9

    # ****************************************************************
    # Solve
    # ****************************************************************
    LpSolverDefault.msg = 1
    dk_solve.solve()

    # ****************************************************************
    # Results
    # ****************************************************************

    print("Solution Status: " + LpStatus[dk_solve.status])

    # Get Selected Player IDs
    team = [v.name for v in dk_solve.variables() if v.varValue == 1]

    return team